# Solution to 538 Riddler, 2020-01-03

This notebook contains the solution to "Spelling Bee"

See https://fivethirtyeight.com/features/can-you-solve-the-vexing-vexillology/

In [1]:
# Read the word list
word_list = open('../538_word_list.txt', 'r').read().split('\n')
print(len(word_list))
word_list[:10]

172820


['aa',
 'aah',
 'aahed',
 'aahing',
 'aahs',
 'aal',
 'aalii',
 'aaliis',
 'aals',
 'aardvark']

We are not interested in any words with a length of < 4 (zero score) and > 7 distinct characters (can't be generated by the lattice), so let's drop all those:

In [2]:
word_list = [x for x in word_list if len(x) >= 4 and len(list(set(x))) <= 7]
len(word_list)

98141

Now generate a dictionary with a score for each word.

Score is calculated as 1, if length = 4; length, if length > 4; add a bonus of 7 if seven distinct characters are used.

This will allow us constant-time score lookups, which will speed up further processing.

In [3]:
def get_score(word):
    if len(word) < 4:
        return(0)
    if len(word) == 4:
        return(1)
    else:
        bonus = 7 if len(list(set(word))) == 7 else 0
        return(len(word) + bonus)
    
word_score_dict = {x: get_score(x) for x in word_list}

Now, each word is generated by a set of letters from the honeycomb lattice. Letters from the honeycomb lattice may be repeated in order to generate a word, so let's create a "minimal set" of letters for each word - take all the letters of the word, remove duplicates, and sort them alphabetically. Multiple different words may have the same minimal set of letters. If we add up the scores of all the words with a specific minimal set of letters, we can go easily from a set of letters to a total score without having to search through the entire word list. Let's do that, and generate a dictionary:

In [4]:
minimal_set_dict = {}

for word in word_score_dict.keys():
    minimal_letters = ''.join(sorted(list(set(word))))
    if minimal_letters not in minimal_set_dict.keys():
        minimal_set_dict[minimal_letters] = get_score(word)
    else:
        minimal_set_dict[minimal_letters] = minimal_set_dict[minimal_letters] + get_score(word)
            

Let's generate a set of candidate honeycomb lattices. Each lattice must have a pangram, so the set of candidate lattices is the same as the set of pangram words. Many pangram words can be potentially be generated from the same lattice, so we again remove duplicate letters and sort alphabetically

In [5]:
candidate_lattices = list(set([''.join(sorted(list(set(x)))) for x in word_score_dict.keys() if len(list(set(x))) == 7]))

len(candidate_lattices)

15599

In [6]:
candidate_lattices[:10]

['eorstuw',
 'cenostu',
 'bdelrsu',
 'deilosz',
 'adeiqru',
 'befirst',
 'abceilt',
 'eimnprt',
 'bikmnos',
 'deilnvy']

Each candidate lattice is actually seven lattices: one with each letter at center. To calculate the total score of a particular lattice, we generate all possible mimimal sets of letters and score them. There has to be between one and six letters (plus the center letter) in a minimal set, and order doesn't matter, so the possible number of miminal sets is ${6 \choose 1} + {6 \choose 2} + ... + {6 \choose 6} = 2^6 - 1 = 63$. This isn't a large number, and we just need to do so many dictionary lookups to get the total score of a lattice.

In [7]:
from itertools import chain, combinations

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1))

def score_lattice(lattice_letters):
    # assume the first letter is the center
    score = 0
    for element in powerset(list(lattice_letters[1:])):
        cur_word = ''.join(sorted([lattice_letters[0]] + list(element)))
        score = score + minimal_set_dict.get(cur_word, 0)
    return score
    

Now let's score all lattices:

In [8]:
def next_lattice_gen():
    for lattice in candidate_lattices:
        for i in range(7):
            yield lattice[i] + lattice.replace(lattice[i], '')
            
lattice_list = next_lattice_gen()

In [9]:
res = [[''.join(x), score_lattice(x)] for x in lattice_list]
# This takes < 10 seconds on my laptop; its really quick

In [10]:
res[:10]

[['eorstuw', 2354],
 ['oerstuw', 1672],
 ['reostuw', 2402],
 ['seortuw', 2454],
 ['teorsuw', 2328],
 ['ueorstw', 1213],
 ['weorstu', 564],
 ['cenostu', 1175],
 ['ecnostu', 1760],
 ['nceostu', 1517]]

In [11]:
from operator import itemgetter

Highest-scoring lattice without 's': (Remember, first letter is the center)

In [12]:
sorted([x for x in res if 's' not in x[0]], key = itemgetter(1), reverse = True)[0]

['raegint', 3898]

Highest-scoring lattice (overall):

In [13]:
sorted(res, key = itemgetter(1), reverse = True)[0]

['eainrst', 8681]

Lowest-scoring lattice overall:

In [14]:
sorted(res, key = itemgetter(1))[0]

['xcinopr', 14]